In [1]:
from unicodedata import name
import matplotlib.pyplot as plt
from sqlalchemy import false
from torch import nn, unsqueeze
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from torchvision import transforms
from torchvision.models.detection.mask_rcnn import maskrcnn_resnet50_fpn
import torchvision
import nibabel as nib
import napari
import os
#from vnet.vnet import VNet
#from vnet.transformations import ResizeImage
import random
from datetime import date
from collections import OrderedDict
from monai.losses import FocalLoss


c:\Users\vegaarellano\Documents\Gitlab\instancesegmentation\pytorch-img-seg\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
import torch.nn as nn
import torch
from torch import autograd
from PIL import Image
from torchvision import transforms
import numpy as np
#network with 3 conv-and-deconv steps used in paper

class one_step_conv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(one_step_conv, self).__init__()
        self.conv = nn.Sequential(
            # Level 1
            nn.Conv2d(in_channels=in_ch, out_channels=out_ch, kernel_size=3, padding=1),            
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            # Level 2            
            nn.Conv2d(in_channels=out_ch, out_channels=out_ch, kernel_size=3, padding=1),            
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            #nn.Dropout2d(p=0.2)             
        )

    def forward(self, input):
        return self.conv(input)

# class one_step_conv(nn.Module):
#     def __init__(self, in_ch, out_ch):
#         super(one_step_conv, self).__init__()
#         self.conv = nn.Sequential(            
#             nn.Conv2d(in_channels=in_ch, out_channels=out_ch, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.BatchNorm2d(out_ch),
#             nn.Dropout(p = 0.1)
            
#         )

#     def forward(self, input):
#         return self.conv(input)

class UnetDeep3(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(UnetDeep3, self).__init__()
        base_filter_num = 64
        self.conv_down_1 = one_step_conv(in_ch, base_filter_num)
        self.pool1 = nn.MaxPool2d(2)
        self.conv_down_2 = one_step_conv(base_filter_num, base_filter_num*2)
        self.pool2 = nn.MaxPool2d(2)
        self.conv_down_3 = one_step_conv(base_filter_num*2, base_filter_num*4)
        self.pool3 = nn.MaxPool2d(2)
        self.conv_bottom = one_step_conv(base_filter_num*4, base_filter_num*8)
        
        
        self.upsample_1 = nn.ConvTranspose2d(base_filter_num*8, base_filter_num*4, kernel_size=2, stride=2)
        self.conv_up_1 = one_step_conv(base_filter_num*8, base_filter_num*4)
        self.upsample_2 = nn.ConvTranspose2d(base_filter_num*4, base_filter_num*2, kernel_size=2, stride=2)
        self.conv_up_2 = one_step_conv(base_filter_num*4, base_filter_num*2)
        self.upsample_3 = nn.ConvTranspose2d(base_filter_num*2, base_filter_num, kernel_size=2, stride=2)
        self.conv_up_3 = one_step_conv(base_filter_num*2, base_filter_num)
        self.conv_out = nn.Conv2d(base_filter_num, out_ch, 1)

    def forward(self, x):
        ###down########        
        down_1 = self.conv_down_1(x)
        pool_1 = self.pool1(down_1)
        down_2 = self.conv_down_2(pool_1)
        pool_2 = self.pool2(down_2)
        down_3 = self.conv_down_3(pool_2)
        pool_3 = self.pool3(down_3)
        bottom = self.conv_bottom(pool_3)

        up_1 = self.upsample_1(bottom)
        merge1 = torch.cat([up_1, down_3], dim=1)
        #print(merge1.shape)
        up_1_out = self.conv_up_1(merge1)
        up_2 = self.upsample_2(up_1_out)
        merge2 = torch.cat([up_2, down_2], dim=1)
        #print(merge2.shape)
        up_2_out = self.conv_up_2(merge2)
        up_3 = self.upsample_3(up_2_out)
        merge3 = torch.cat([up_3, down_1], dim=1)
        #print(merge3.shape)
        up_3_out = self.conv_up_3(merge3)
        end_out = self.conv_out(up_3_out)
        #print(end_out.shape)
        #return end_out
        # Applying classification in the output layer
        out = nn.Sigmoid()(end_out)
        #out = nn.Softmax()(end_out)
        #print(out.shape)
        #out = nn.Sigmoid(end_out)

        return out



In [21]:
# 
torch.manual_seed(200)

with torch.no_grad():
    x = torch.randn((1,1,224,224))
    y = torch.randn((1,1,224,224)) # 
    y = (y > 0.5).float()

    model = UnetDeep3(in_ch=1, out_ch=1)
    prediction = model(x)
    print(prediction.shape)
    print(torch.max(prediction))
    print(torch.min(prediction))
    print(prediction[0,0,100,100:110])
    print(y[0,0,100,100:110])
    print(torch.sigmoid(torch.max(prediction)))
    print(torch.sigmoid(torch.min(prediction)))


torch.Size([1, 1, 224, 224])
tensor(0.8233)
tensor(0.0755)
tensor([0.3213, 0.5990, 0.3876, 0.4177, 0.3085, 0.4112, 0.5947, 0.5021, 0.5149,
        0.5263])
tensor([0., 0., 0., 1., 1., 1., 0., 0., 0., 0.])
tensor(0.6949)
tensor(0.5189)


In [23]:
#  Trying with Focal Loss
loss = FocalLoss()
loss_value = loss(prediction, y)
loss_value.item()


0.2711390554904938

In [10]:
#  Trying with WBCE
loss = nn.BCELoss(weight=torch.FloatTensor([.05]))
loss_value = loss(prediction, y)
loss_value.item()


0.03329070284962654

In [39]:
from torch.autograd import Variable
# Example of loss functions
torch.manual_seed(100)
x = Variable(torch.randn(1,1,224,224))
y = Variable(torch.FloatTensor(1,1,224,224).random_(2))
print(x)
print(y)


tensor([[[[ 0.1268,  1.3564,  0.5632,  ...,  1.8463,  0.9502, -0.3396],
          [-0.6599, -0.9520, -0.4073,  ...,  1.6103, -1.6023, -1.3940],
          [-0.2074,  0.8496, -1.0217,  ...,  0.1431, -0.3378,  0.8120],
          ...,
          [ 1.4933,  0.9621, -0.4328,  ...,  2.3392, -0.3927, -1.0324],
          [-0.5816,  0.2777,  1.3490,  ...,  2.5849,  0.5224,  0.9784],
          [ 1.2349, -0.5501,  0.4484,  ...,  0.9272,  0.5862, -1.0449]]]])
tensor([[[[1., 0., 1.,  ..., 0., 1., 1.],
          [0., 0., 0.,  ..., 0., 1., 1.],
          [0., 0., 0.,  ..., 1., 0., 0.],
          ...,
          [0., 0., 1.,  ..., 0., 0., 0.],
          [1., 0., 1.,  ..., 1., 0., 0.],
          [0., 1., 0.,  ..., 0., 0., 0.]]]])


In [40]:

# double the loss for class 1 ( 3 classes in the loss)
# Probability Background: 95% => Stroke: 5%
# Weights inverted for BCE function
class_weight = torch.FloatTensor([0.05])
class_weight

tensor([0.0500])

In [41]:
# double the loss for last sample
element_weight = torch.FloatTensor([1.0]*9 + [2.0]).view(-1, 1)
print(element_weight)
element_weight = element_weight.repeat(1, 3)
element_weight

tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [2.]])


tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [2., 2., 2.]])

In [42]:
bce_criterion = nn.BCEWithLogitsLoss(weight=None, reduce=False)
bce_loss = bce_criterion(x, y)
print(bce_loss)

bce_criterion_class = nn.BCEWithLogitsLoss(weight=class_weight, reduce=False)
bce_loss_class = bce_criterion_class(x, y)
print(bce_loss_class)

tensor([[[[0.6317, 1.5856, 0.4507,  ..., 1.9928, 0.3269, 0.8773],
          [0.4167, 0.3264, 0.5101,  ..., 1.7925, 1.7858, 1.6156],
          [0.5948, 1.2056, 0.3075,  ..., 0.6241, 0.5384, 1.1794],
          ...,
          [1.6959, 1.2857, 0.9328,  ..., 2.4312, 0.5160, 0.3046],
          [1.0257, 0.8416, 0.2307,  ..., 0.0727, 0.9881, 1.2975],
          [1.4902, 1.0056, 0.9423,  ..., 1.2606, 1.0286, 0.3014]]]])
tensor([[[[0.0316, 0.0793, 0.0225,  ..., 0.0996, 0.0163, 0.0439],
          [0.0208, 0.0163, 0.0255,  ..., 0.0896, 0.0893, 0.0808],
          [0.0297, 0.0603, 0.0154,  ..., 0.0312, 0.0269, 0.0590],
          ...,
          [0.0848, 0.0643, 0.0466,  ..., 0.1216, 0.0258, 0.0152],
          [0.0513, 0.0421, 0.0115,  ..., 0.0036, 0.0494, 0.0649],
          [0.0745, 0.0503, 0.0471,  ..., 0.0630, 0.0514, 0.0151]]]])


In [ ]:




multi_criterion_class = nn.MultiLabelSoftMarginLoss(weight=class_weight, reduce=False)

bce_criterion_element = nn.BCEWithLogitsLoss(weight=element_weight, reduce=False)
multi_criterion_element = nn.MultiLabelSoftMarginLoss(weight=element_weight, reduce=False)




bce_loss_class = bce_criterion_class(x, y)


bce_loss_element = bce_criterion_element(x, y)
multi_loss_element = multi_criterion_element(x, y)

print(bce_loss - multi_loss)
print(bce_loss_class - multi_loss_class)
print(bce_loss_element - multi_loss_element)

In [19]:
import torch
import torchvision
from torchmetrics.functional import precision_recall
from torchmetrics import F1Score

torch.manual_seed(200)
preds  = torch.randn([1,1,25,25])
target = torch.randn([1,1,25,25])
target = (target > 0.5).int()
preds = (preds > 0.5).int()
preds = preds.reshape(-1)
target = target.reshape(-1)


print("============= Samples: ============")
print(target[0:20])
print(preds[0:20])
print("============= Precision recall: ============")
result = precision_recall(preds, target, average='macro', num_classes=2)
print(result)
result = precision_recall(preds, target, average='micro')
print(result)
result = precision_recall(preds, target, average='weighted', num_classes=2)
print(result)

print("============= F1 Score ============")
f1 = F1Score(num_classes=2)
print(f1(preds, target))

print("============= F1 Score ============")
f1 = F1Score()
print(f1(preds, target))

============= Samples: ============
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=torch.int32)
tensor([0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1],
       dtype=torch.int32)
============= Precision recall: ============
(tensor(0.4806), tensor(0.4790))
(tensor(0.5712), tensor(0.5712))
(tensor(0.5894), tensor(0.5712))
============= F1 Score ============
tensor(0.5712)
============= F1 Score ============
tensor(0.5712)
